# Практика. API Headhunter

Проведем анализ вакансий для Data Science. Документация API hh: https://api.hh.ru/openapi/redoc

In [45]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
from time import sleep
from collections import Counter

Запросы к API всегда выстраиваются в таком формате: 

URL API + нужный раздел + API-ключ (по требованию) + параметры

https://api.hh.ru/vacancies?text=data%20science&only_with_salary=true&per_page=100

Что в этом запросе?

`https://api.hh.ru/vacancies` – запрос к БД с вакансиями \
`?text=data%20science` – прицепляем с помощью **?** первый параметр `text` с поиском по тексу вакансий \
`&only_with_salary=true` – все следующие параметры прицепляем через **&**. Ищем вакансии только с указанной з/п \
`&per_page=100` – используем пагинацию

In [8]:
vacancies = []

for i in range(0, 2):
    vac = requests.get(f"https://api.hh.ru/vacancies?text=data%20science&only_with_salary=true&per_page=100&page={i}")
    vacancies.extend(json.loads(vac.text)['items'])

In [10]:
vacancies[0]

{'id': '89601541',
 'premium': False,
 'name': 'Junior Backend Developer (Java)',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '2',
  'name': 'Санкт-Петербург',
  'url': 'https://api.hh.ru/areas/2'},
 'salary': {'from': 60000, 'to': 120000, 'currency': 'RUR', 'gross': True},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Санкт-Петербург',
  'street': 'Аптекарский проспект',
  'building': '4',
  'lat': 59.969533,
  'lng': 30.318276,
  'description': None,
  'raw': 'Санкт-Петербург, Аптекарский проспект, 4',
  'metro': {'station_name': 'Петроградская',
   'line_name': 'Московско-Петроградская',
   'station_id': '15.215',
   'line_id': '15',
   'lat': 59.966389,
   'lng': 30.311293},
  'metro_stations': [{'station_name': 'Петроградская',
    'line_name': 'Московско-Петроградская',
    'station_id': '15.215',
    'line_id': '15',
    'lat': 59.966389,
    'lng': 30.311293}],
  'id': '3942995'},
 'response_url': None,
 'sor

In [9]:
for v in vacancies:
    print(v['name'], v['salary']['from'], v['salary']['to'], v['area']['name'], v['url'])

Junior Backend Developer (Java) 60000 120000 Санкт-Петербург https://api.hh.ru/vacancies/89601541?host=hh.ru
Аналитик данных (Data Science) 50000 100000 Иннополис https://api.hh.ru/vacancies/89380163?host=hh.ru
Специалист по работе с клиентами 40000 None Чебоксары https://api.hh.ru/vacancies/87623047?host=hh.ru
C++ developer to Berlin 7500 10834 Германия https://api.hh.ru/vacancies/89443252?host=hh.ru
Делопроизводитель 50000 None Москва https://api.hh.ru/vacancies/89215196?host=hh.ru
Data-science-специалист 40000 100000 Москва https://api.hh.ru/vacancies/89072255?host=hh.ru
Backend-разработчик 120000 None Ростов-на-Дону https://api.hh.ru/vacancies/89607557?host=hh.ru
Специалист по работе с физическими лицами (без продаж, без поиска клиентов) 50000 None Саратов https://api.hh.ru/vacancies/87710180?host=hh.ru
АККРЕДИТОВАННАЯ КОМПАНИЯ Начинающий Java разработчик 40000 None Ростов-на-Дону https://api.hh.ru/vacancies/89291378?host=hh.ru
Senior Customer Service Coordinator 160000 None Москва

Соберем все ссылки на вакансии, чтобы вытащить оттуда описание вакансии и ключевые навыки.

In [11]:
links = []

for v in vacancies:
    links.append(v['url'])

In [37]:
test = requests.get(links[0])
test0 = json.loads(test.text)
test0

{'id': '89601541',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Junior Backend Developer (Java)',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '2',
  'name': 'Санкт-Петербург',
  'url': 'https://api.hh.ru/areas/2?host=hh.ru'},
 'salary': {'from': 60000, 'to': 120000, 'currency': 'RUR', 'gross': True},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Санкт-Петербург',
  'street': 'Аптекарский проспект',
  'building': '4',
  'lat': 59.969533,
  'lng': 30.318276,
  'description': None,
  'raw': 'Санкт-Петербург, Аптекарский проспект, 4',
  'metro': {'station_name': 'Петроградская',
   'line_name': 'Московско-Петроградская',
   'station_id': '15.215',
   'line_id': '15',
   'lat': 59.966389,
   'lng': 30.311293},
  'metro_stations': [{'station_name': 'Петроградская',
    'line_name': 'Московско-Петроградская',
    'station_id': '15.215',
    'line_id': '15',
    'lat': 59.966389,

In [19]:
test0['description']

'<p>Мы IT-компания из Санкт-Петербурга, которая занимается разработкой мобильных приложений, веб-порталов, высоконагруженных сервисов, облачных решений, Gamedev-проектов и Data Science. Мы активно развиваемся, в нашей команде уже более 100 человек.</p> <p>Сейчас мы ищем <strong>Java разработчика,</strong> который хочет участвовать в интересных проектах, прокачивать свои навыки и расти вместе с нами.</p> <p> </p> <p><strong>Обязанности:</strong></p> <ul> <li>Разработка backend-программного обеспечения для веб-сайтов и мобильных приложений</li> <li>Проектирование и реализация баз данных MySQL / SQLite / PostgreSQL</li> <li>Проектирование и разработка REST API</li> </ul> <p><strong>Требования:</strong></p> <ul> <li>Умение работать с Java и фреймоворком Spring</li> <li>Умение работать с MySQL / PostgreSQL, REST API</li> </ul> <p><strong>Условия:</strong></p> <ul> <li>Гибкий график (согласовывается индивидуально)</li> <li>Решение интересных задач</li> <li>Работа в молодой команде</li> <li>Р

In [28]:
test0['key_skills']

[{'name': 'Java SE'},
 {'name': 'Spring Framework'},
 {'name': 'PostgreSQL'},
 {'name': 'ООП'}]

In [27]:
[i['name'] for i in test0['key_skills']]

['Java SE', 'Spring Framework', 'PostgreSQL', 'ООП']

Соберем в одну таблицу.

In [81]:
info = []

for l in tqdm(links):
    v = requests.get(l)
    v_loaded = json.loads(v.text)
    
    if v_loaded['salary']['currency'] == 'RUR':
    
        descr = v_loaded['description']
        skills = [i['name'] for i in v_loaded['key_skills']]
        money_from = v_loaded['salary'].get('from', np.nan)
        money_to = v_loaded['salary'].get('to', np.nan)
        url = v_loaded['alternate_url']

        info.append([descr, skills, money_from, money_to, url])
        sleep(3)

100%|█████████████████████████████████████████| 163/163 [06:29<00:00,  2.39s/it]


In [82]:
df = pd.DataFrame(info, columns=['description', 'key_skills', 'salary_from', 'salary_to', 'url'])

In [83]:
df

,description,key_skills,salary_from,salary_to,url
0,"<p>Мы IT-компания из Санкт-Петербурга, которая...","[Java SE, Spring Framework, PostgreSQL, ООП]","60,000.00","120,000.00",https://hh.ru/vacancy/89601541
1,<p>Мы создали<strong> RANKS.pro</strong> — Сис...,"[Python, Математическая статистика, SQL, Panda...","50,000.00","100,000.00",https://hh.ru/vacancy/89380163
2,<p><em><strong>IDF Eurasia </strong></em>специ...,[],"40,000.00",NaN,https://hh.ru/vacancy/87623047
3,<p><strong>Karpov.courses </strong>— молодая E...,"[Документооборот, Работа с большим объемом инф...","50,000.00",NaN,https://hh.ru/vacancy/89215196
4,"<strong>O2 Trading - ищем, проверяем, внедряем...","[Python, SQL, Анализ данных, Работа с большим ...","40,000.00","100,000.00",https://hh.ru/vacancy/89072255
...,...,...,...,...,...
117,<p>Pygmalion AI — это передовая AI-платформа д...,"[Python, Аналитическое мышление, Data Analysis...","400,000.00",NaN,https://hh.ru/vacancy/88412207
118,<p>Pygmalion AI — это передовая AI-платформа д...,"[Python, Аналитическое мышление, Data Analysis...","400,000.00",NaN,https://hh.ru/vacancy/88412212
119,<p><strong>Про нас:</strong></p> <p>OTUS сегод...,"[Расчет заработной платы, Расчет зарплаты, Бух...",NaN,"80,000.00",https://hh.ru/vacancy/89142460
120,<p>Pygmalion AI — это передовая AI-платформа д...,"[Python, Аналитическое мышление, Data Analysis...","400,000.00",NaN,https://hh.ru/vacancy/88412209


In [84]:
df['key_skills'].sum()

['Java SE',
 'Spring Framework',
 'PostgreSQL',
 'ООП',
 'Python',
 'Математическая статистика',
 'SQL',
 'Pandas',
 'Машинное обучение',
 'Документооборот',
 'Работа с большим объемом информации',
 'Деловая переписка',
 'Грамотная речь',
 'amoCRM',
 'Делопроизводство',
 'Электронный документооборот',
 'Python',
 'SQL',
 'Анализ данных',
 'Работа с большим объемом информации',
 'Базы данных',
 'Аналитическое мышление',
 'Работа с базами данных',
 'Математическая статистика',
 'Python',
 'PostgreSQL',
 'Docker',
 'SQL',
 'Git',
 'FastAPI',
 'Backend',
 'MS SQL',
 'Базы данных',
 'ООП',
 'SOLID',
 'Работа в команде',
 'Грамотная речь',
 'Телефонные переговоры',
 'Работа с базами данных',
 'Ориентация на результат',
 'Git',
 'Java Core',
 'Spring',
 'Java',
 'Hibernate',
 'Apache Maven',
 'PostgreSQL',
 'Hibernate ORM',
 'Web',
 'MySQL',
 'MongoDB',
 'Linux',
 'ООП',
 'Teamplayer',
 'Time management',
 'Negotiation skills',
 'Analytical skills',
 'Sales Skills',
 'JavaScript',
 'Bootstrap

In [85]:
Counter(df['key_skills'].sum()).most_common(15)

[('Python', 37),
 ('SQL', 27),
 ('Аналитическое мышление', 16),
 ('Data Analysis', 12),
 ('Анализ данных', 11),
 ('Работа с базами данных', 11),
 ('ETL', 11),
 ('MySQL', 10),
 ('Git', 9),
 ('Английский язык', 9),
 ('Бизнес-анализ', 9),
 ('Agile Project Management', 9),
 ('Грамотная речь', 8),
 ('Big Data', 8),
 ('NLP', 8)]

In [86]:
pd.options.display.float_format = '{:,.2f}'.format

df[['salary_from', 'salary_to']].describe()

,salary_from,salary_to
count,106.00,63.00
mean,"150,718.40","234,171.43"
std,"122,098.93","166,547.08"
min,"30,000.00","30,000.00"
25%,"50,000.00","100,000.00"
50%,"100,000.00","150,000.00"
75%,"200,000.00","390,000.00"
max,"400,000.00","550,000.00"


In [87]:
with pd.option_context('display.max_colwidth', None):
    display(df[df['salary_from'] == df['salary_from'].max()])

description  \
68                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     <p>Pygmalion AI — это передовая AI-платформа для создания, публикации и продвижения контента в социальных медиа. Для достижения этой цели мы ищем Старшего инженера по данным, который будет играть ключевую роль во внедрении практик DataOps для эффективного управления данными социальных медиа. В этой роли подразумевается контроль за всем жизненным циклом данных, обеспечение целостности данных и расширение возможностей нашей AI-платформы для обеспечения исключительного пользовательского опыта.</p> <p>Ключевые обязанности:</p> <ul> <li>Проектирование, разработка и поддержание надежных и масштабируемых конвейеров данных для сбора, обработки и хранения данных из различных источников социальных медиа и пользовательских взаимодействий.</li> <li>Проектирование хранилища данных.</li> <li>Реализация строгих проверок качества данных и процессов валидации для поддержания целостности, точности и надежности данных социальных медиа, используемых нашими AI-моделями.</li> <li>Автоматизация процессов извлечения, трансформации и загрузки (ETL) для оптимизации загрузки и трансформации данных, уменьшения ручного вмешательства и повышения эффективности.</li> <li>Непрерывный мониторинг и оптимизация конвейеров данных для улучшения скорости, надежности и масштабируемости, обеспечивая бесшовную работу нашего AI-ассистента.</li> <li>Тесное сотрудничество с инженерами машинного обучения и межфункциональными командами для понимания требований к данным и предоставления необходимой инфраструктуры данных для разработки и обучения моделей.</li> <li>Проведение практик управления данными, гарантирование конфиденциальности данных, безопасности и соответствия соответствующим нормативам, включая GDPR, в контексте данных социальных медиа.</li> <li>Установление показателей производительности и внедрение решений мониторинга для выявления и устранения узких мест или аномалий в конвейере данных.</li> <li>Сотрудничество с аналитиками данных и бизнес-командами для проектирования интерактивных панелей инструментов, которые обеспечивают принятие решений на основе данных.</li> <li>Разработка и поддержка датамартов и панелей инструментов, предоставляющих оперативные аналитические данные по данным социальных медиа.</li> <li>Быть в курсе появляющихся технологий, инструментов и фреймворков в области данных, оценивая их потенциал для улучшения процессов инженерии данных.</li> </ul> <p> </p> <p>Требования к кандидату:</p> <ul> <li>Степень бакалавра или магистра по компьютерным наукам, инженерии данных или смежной области.</li> <li>Доказанный опыт работы в области инженерии данных, с акцентом на процессы ETL, разработку конвейеров данных и обеспечение качества данных.</li> <li>Высокая профессиональная компетенция в языках программирования, таких как Python, SQL, а также знание библиотек и фреймворков инженерии данных.</li> <li>Опыт работы с облачными решениями для хранения и обработки данных, такими как AWS,

## Задание 1

Найдите все вакансии, в которых тип занятости "Волонтер".
В этом вам поможет поле "employment", в которое подставляются значения из общих словарей: https://api.hh.ru/dictionaries

In [ ]:
# ваш код

## Задание 2

Найдите, сколько вакансий на hh было опубликовано за эту неделю.

In [ ]:
# ваш код

## Задание 3

Найдите все вакансии, которые сейчас открыты у вашей компании (учебного заведения / интересующей вас компании). Загрузите данные в таблицу и сделайте описательную статистику заработных плат в вакансиях с помощью метода describe().

В этом вам поможет модуль "Поиск работодателя": https://api.hh.ru/openapi/redoc#tag/Rabotodatel/operation/search-employer

In [ ]:
# ваш код